In [4]:
%load_ext nb_black
import pickle
import cv2
import glob
import pandas as pd
import numpy as np
from collections import defaultdict
import sys
import os

sys.path.append("../src")
from dtw import *
from main import normalize_traj
import matplotlib.pyplot as plt
from ipywidgets import interact

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [5]:
path_df = pd.DataFrame({"path": glob.glob("../data/template/*/*/*.pickle")})
path_df["group"] = path_df["path"].agg(lambda t: t.split("/")[-2])
path_df["template"] = path_df["path"].agg(lambda t: t.split("/")[-3])
h, w = (720, 1280)

<IPython.core.display.Javascript object>

In [6]:
def extract_data(pathlist, normalize=False):
    data = []
    for path in sorted(pathlist):
        with open(path, "rb") as f:
            tmp = pickle.load(f)
        data.append([tmp.landmark[8].x, tmp.landmark[8].y])
    data = np.asarray(data)
    if normalize:
        min_x = np.min(data.T[0])
        max_x = np.max(data.T[0])
        min_y = np.min(data.T[1])
        max_y = np.max(data.T[1])
        data -= np.array([min_x, min_y])
        data /= np.array([max_x - min_x, max_y - min_y])
    return data


templates = sorted(path_df["template"].unique())


@interact(i=(0, len(templates) - 1))
def select_t(i=0):
    template = templates[i]
    t_df = path_df.query("template==@template")
    groups = sorted(t_df["group"].unique())

    @interact(j=(0, len(groups) - 1))
    def select_g(j=0, normalize=False):
        group = groups[j]
        g_df = t_df.query("group==@group")
        im = np.zeros([h, w, 3], dtype=np.uint8)
        data = extract_data(g_df["path"], normalize)
        for x, y in data:
            cv2.circle(im, (int(w * x), int(h * y)), 5, (255, 0, 0), -1)
        plt.figure(figsize=(10, 6))
        plt.imshow(im[:, :, ::-1])

interactive(children=(IntSlider(value=0, description='i', max=5), Output()), _dom_classes=('widget-interact',)…

<IPython.core.display.Javascript object>

In [7]:
normalize = True
traj = defaultdict(list)
for template in templates:
    t_df = path_df.query("template==@template")
    groups = sorted(t_df["group"].unique())
    for group in groups[1:]:
        g_df = t_df.query("group==@group")
        traj[template].append(extract_data(g_df["path"], normalize))

test_traj = defaultdict(list)
for template in templates:
    t_df = path_df.query("template==@template")
    groups = sorted(t_df["group"].unique())
    for group in groups[0:1]:
        g_df = t_df.query("group==@group")
        test_traj[template].append(extract_data(g_df["path"], normalize))


<IPython.core.display.Javascript object>

In [8]:
scores = {}

for t_key,t_val in test_traj.items():
    scores[t_key]=defaultdict(list)
    for key,val_list in traj.items():
        for val in val_list:
            dp, path = dtw(t_val[0],val)
            scores[t_key][key].append(dp[-1][-1])


<IPython.core.display.Javascript object>

In [9]:
key_list = list(test_traj.keys())


@interact(t_key=key_list)
def select(t_key=key_list[0]):
    score = scores[t_key]
    plt.figure(figsize=(12, 6))
    for k, val in score.items():
        plt.scatter(range(len(val)), val, label=k)
    plt.legend()

interactive(children=(Dropdown(description='t_key', options=('a', 'e', 'i', 'nadare', 'o', 'u'), value='a'), O…

<IPython.core.display.Javascript object>